In [1]:
from undulator import InsertionDevice, ElectronBeam, Beamline

In [7]:
Keff = 1.38
lamda_w = 18e-3
E = 3e9

achr3 = InsertionDevice(Kmax=Keff, lamda_w=lamda_w, Np=111, L=2)
r3 = ElectronBeam(energy=E,
                  emittance={'x': 340e-12, 'y': 8e-12},
                  beta={'x': 9, 'y': 4.7},
                  espread=0.8e-3
                 )
nanomax = Beamline(undulator=achr3, electronbeam=r3)

n=1
print(nanomax)
print(f'Brightness => {nanomax.brightness(n)}')
print(f'Max brightness => {nanomax.brightness_max(n)}')
print(f'Ratio => {nanomax.brightness_max(n) / nanomax.brightness(n)}')

InsertionDevice:: Kmax=1.38 Period=18.0 mm Ncells=111
ElectronBeam:: 3.0 GeV ex=3.4e-10 ey=8e-12 bx=9 by=4.7
Brightness => 6.213443221613991e+21
Max brightness => 3.378106476702498e+22
Ratio => 5.436770492971541
